I would suggest to just utilize the C headers once they are finished and steal those declarations as suggestes in the SAFE_dataParser.ipynb

then we'd have all the data types and function declarations as we need them (with little editing) to use them for cffi

![image of preprocessing](preprocessing.png)

---

#### after having those first method declarations the following concept parser prints out 'empty' methods for a python class wanting to utilizing those methods

In [1]:
from cffi import FFI
ffi = FFI()

In [2]:
functionsFromCompiler='''void access_container_refresh_access_info(App const* app, void* user_data, void (*o_cb)(void* user_data, FfiResult const* result));
void access_container_fetch(App const* app, void* user_data, void (*o_cb)(void* user_data, FfiResult const* result, FfiContainerPermissions const* container_perms, uintptr_t container_perms_len));
void access_container_get_container_mdata_info(App const* app, char const* name, void* user_data, void (*o_cb)(void* user_data, FfiResult const* result, FfiMDataInfo const* mdata_info));
void cipher_opt_new_plaintext(App const* app, void* user_data, void (*o_cb)(void* user_data, FfiResult const* result, CipherOptHandle handle));
void app_unregistered(uint8_t const* bootstrap_config, uintptr_t bootstrap_config_len, void* user_data, void (*o_disconnect_notifier_cb)(void* user_data), void (*o_cb)(void* user_data, FfiResult const* result, App* app));
void app_registered(char const* app_id, FfiAuthGranted const* auth_granted, void* user_data, void (*o_disconnect_notifier_cb)(void* user_data), void (*o_cb)(void* user_data, FfiResult const* result, App* app));
'''

In [3]:
import re

In [4]:
startOfFunction=re.compile('.*?\(')

In [5]:
parameters=re.compile('\(.*\)')

In [6]:
functions=re.compile(', void \(\*.*?\)\(.*?\)')

In [7]:
def getParameterDict(callParameters):
    mydict={}
    for onePar in callParameters:
        datatype,name = onePar.split()
        mydict[name]=datatype
    return mydict

In [8]:
def printDefinitions(callParameters):
    for onekey in callParameters:
        if callParameters[onekey] == 'char*':
            print("    %s = ffi.new('char[]',self.toByteIfString(%s,encoding))" % (onekey, onekey))
        if callParameters[onekey] == 'void*':
            print("""    if %s:
        %s = ffi.new_handle(%s)
    else:
        %s = ffi.NULL""" % (onekey, onekey, onekey, onekey))

In [9]:
def printFunctionCall(callParameters,callbackfunctions,functionName):
    tempstring='    self.lib.' + functionName + '('
    
    for onepar in callParameters:
        tempstring += onepar + ', '
    for onepar in callbackfunctions:
        tempstring += onepar + ', '
    
    tempstring = tempstring[:-2] + ')'
    
    print(tempstring)

In [10]:
def printMethodDefinition(callParameters,callbackfunctions,functionName):
    
    #first the declaration line
    tempstring='def ' + functionName + '(self, '
    for onepar in callParameters:
        if callParameters[onepar] == 'void*':
            tempstring += onepar + '=None, '
        else:
            tempstring += onepar + ', '
    for onepar in callbackfunctions:
        tempstring += onepar + '=None, '
    tempstring += "encoding='utf-8')"
    print(tempstring)
    
    #now the doc string
    tempstring='    """ '
    for onepar in callParameters:
        if callParameters[onepar] == 'void*':
            tempstring += '[any]' + ', '
        elif callParameters[onepar] == 'char*':
            tempstring += 'string/bytes' + ', '
        else:
            tempstring += callParameters[onepar] + ', '
    for onepar in callbackfunctions:
        tempstring += '[function], '
    tempstring += "[encoding]"
    
    print(tempstring)
    
    tempstring='        '
    for onepar in callParameters:
        tempstring += callParameters[onepar] + ' ' + onepar + ', '
    
    print(tempstring[:-2])
    
    print()
    
    print('        > return values of the callback functions:')
    
    for oneCallback in callbackfunctions:
        tempstring='        ' + str(oneCallback) + ' - '
        for oneParam in callbackfunctions[oneCallback]:
            tempstring += callbackfunctions[oneCallback][oneParam] + ' ' + oneParam + ', '
    print(tempstring[:-2])
    print('    """')
    

In [15]:
def printCallbackDef(callbackDict, name, functionname_for_default_result):
    tempstring='    @ffi.callback("void('
    
    for parameter in callbackDict:
        tempstring+=callbackDict[parameter] + ', '
        
    tempstring = tempstring[:-2]+''')")
    def ''' + name + '('
    
    for parameter in callbackDict:
        tempstring+=parameter + ', '
        
    tempstring = tempstring[:-2]+'''):
        
        if ''' + name + ''':
            ''' + name + '''('''
    
    for parameter in callbackDict:
        tempstring+=parameter + ', '
    
    if 'FfiResult*' in callbackDict.values():
        tempstring = tempstring[:-2]+''')
        else:
            self.defaultFfiResult(result,\'''' + functionname_for_default_result + '''')'''
    else:
        tempstring = tempstring[:-2]+''')
        else:
            pass'''
    
    print(tempstring)

In [12]:
def printWholeDefinition(oneline, parentClassName):
    nameOfFunction=startOfFunction.findall(oneline)[0][5:-1]
    allParameters=parameters.findall(oneline)[0][1:-1]
    callbackfunctions=functions.findall(oneline)
    
    
    
    callbackfunctions=functions.findall(oneline)
    
    CallParameters=allParameters
    for oneCallback in callbackfunctions:
        CallParameters=CallParameters.replace(oneCallback,'')
    CallParameters=CallParameters.replace(' const','').split(',')
    
    paramDict=getParameterDict(CallParameters)
    
    callbackDict={}
    for oneCallback in callbackfunctions:
        callbackDict[re.findall('\(\*.*?\)',oneCallback)[0][2:-1]]=getParameterDict(re.findall('\(.*?\)',oneCallback.replace(' const',''))[-1][1:-1].split(', '))
        
        
    printMethodDefinition(paramDict,callbackDict,nameOfFunction)
    print()
    for oneCallback in callbackDict:
        printCallbackDef(callbackDict[oneCallback],oneCallback,nameOfFunction)
        
        print()
    printDefinitions(paramDict)
    
    print()
    printFunctionCall(paramDict,callbackDict,nameOfFunction)
    
    print()
    print(parentClassName + '.' + nameOfFunction + ' = ' + nameOfFunction)
    print('del(' + nameOfFunction + ')')
    print()
    print()
    
    return callbackDict

In [16]:
for oneline in functionsFromCompiler.splitlines():
    callbackDict=printWholeDefinition(oneline,'Auth')

def access_container_refresh_access_info(self, app, user_data=None, o_cb=None, encoding='utf-8')
    """ App*, [any], [function], [encoding]
        App* app, void* user_data

        > return values of the callback functions:
        o_cb - void* user_data, FfiResult* result
    """

    @ffi.callback("void(void*, FfiResult*)")
    def o_cb(user_data, result):
        
        if o_cb:
            o_cb(user_data, result)
        else:
            self.defaultFfiResult(result,'access_container_refresh_access_info')

    if user_data:
        user_data = ffi.new_handle(user_data)
    else:
        user_data = ffi.NULL

    self.lib.access_container_refresh_access_info(app, user_data, o_cb)

Auth.access_container_refresh_access_info = access_container_refresh_access_info
del(access_container_refresh_access_info)


def access_container_fetch(self, app, user_data=None, o_cb=None, encoding='utf-8')
    """ App*, [any], [function], [encoding]
        App* app, void* user_data

        > retu

In [ ]:
def access_container_refresh_access_info(self, app, user_data=None, o_cb=None, encoding='utf-8')
    """ App*, [any], [function], [encoding]
        App* app, void* user_data

        > return values of the callback functions:
        o_cb - void* user_data, FfiResult* result
    """

    @ffi.callback("void(void*, FfiResult*)")
    def o_cb(user_data, result):
        
        if o_cb:
            o_cb(user_data, result)
        else:
            self.defaultFfiResult(result,'access_container_refresh_access_info')

    if user_data:
        user_data = ffi.new_handle(user_data)
    else:
        user_data = ffi.NULL

    self.lib.access_container_refresh_access_info(app, user_data, o_cb)

Auth.access_container_refresh_access_info = access_container_refresh_access_info
del(access_container_refresh_access_info)

In [ ]:
def access_container_fetch(self, app, user_data=None, o_cb=None, encoding='utf-8')
    """ App*, [any], [function], [encoding]
        App* app, void* user_data

        > return values of the callback functions:
        o_cb - FfiContainerPermissions* container_perms, uintptr_t container_perms_len, void* user_data, FfiResult* result
    """

    @ffi.callback("void(FfiContainerPermissions*, uintptr_t, void*, FfiResult*)")
    def o_cb(container_perms, container_perms_len, user_data, result):
        
        if o_cb:
            o_cb(container_perms, container_perms_len, user_data, result)
        else:
            self.defaultFfiResult(result,'access_container_fetch')

    if user_data:
        user_data = ffi.new_handle(user_data)
    else:
        user_data = ffi.NULL

    self.lib.access_container_fetch(app, user_data, o_cb)

Auth.access_container_fetch = access_container_fetch
del(access_container_fetch)

In [ ]:
def access_container_get_container_mdata_info(self, app, name, user_data=None, o_cb=None, encoding='utf-8')
    """ App*, string/bytes, [any], [function], [encoding]
        App* app, char* name, void* user_data

        > return values of the callback functions:
        o_cb - FfiMDataInfo* mdata_info, void* user_data, FfiResult* result
    """

    @ffi.callback("void(FfiMDataInfo*, void*, FfiResult*)")
    def o_cb(mdata_info, user_data, result):
        
        if o_cb:
            o_cb(mdata_info, user_data, result)
        else:
            self.defaultFfiResult(result,'access_container_get_container_mdata_info')

    name = ffi.new('char[]',self.toByteIfString(name,encoding))
    if user_data:
        user_data = ffi.new_handle(user_data)
    else:
        user_data = ffi.NULL

    self.lib.access_container_get_container_mdata_info(app, name, user_data, o_cb)

Auth.access_container_get_container_mdata_info = access_container_get_container_mdata_info
del(access_container_get_container_mdata_info)


def cipher_opt_new_plaintext(self, app, user_data=None, o_cb=None, encoding='utf-8')
    """ App*, [any], [function], [encoding]
        App* app, void* user_data

        > return values of the callback functions:
        o_cb - CipherOptHandle handle, void* user_data, FfiResult* result
    """

    @ffi.callback("void(CipherOptHandle, void*, FfiResult*)")
    def o_cb(handle, user_data, result):
        
        if o_cb:
            o_cb(handle, user_data, result)
        else:
            self.defaultFfiResult(result,'cipher_opt_new_plaintext')

    if user_data:
        user_data = ffi.new_handle(user_data)
    else:
        user_data = ffi.NULL

    self.lib.cipher_opt_new_plaintext(app, user_data, o_cb)

Auth.cipher_opt_new_plaintext = cipher_opt_new_plaintext
del(cipher_opt_new_plaintext)


def app_unregistered(self, bootstrap_config, bootstrap_config_len, user_data=None, o_disconnect_notifier_cb=None, o_cb=None, encoding='utf-8')
    """ uint8_t*, uintptr_t, [any], [function], [function], [encoding]
        uint8_t* bootstrap_config, uintptr_t bootstrap_config_len, void* user_data

        > return values of the callback functions:
        o_cb - App* app, void* user_data, FfiResult* result
    """

    @ffi.callback("void(void*)")
    def o_disconnect_notifier_cb(user_data):
        
        if o_disconnect_notifier_cb:
            o_disconnect_notifier_cb(user_data)
        else:
            pass

    @ffi.callback("void(App*, void*, FfiResult*)")
    def o_cb(app, user_data, result):
        
        if o_cb:
            o_cb(app, user_data, result)
        else:
            self.defaultFfiResult(result,'app_unregistered')

    if user_data:
        user_data = ffi.new_handle(user_data)
    else:
        user_data = ffi.NULL

    self.lib.app_unregistered(bootstrap_config, bootstrap_config_len, user_data, o_disconnect_notifier_cb, o_cb)

Auth.app_unregistered = app_unregistered
del(app_unregistered)


def app_registered(self, app_id, auth_granted, user_data=None, o_disconnect_notifier_cb=None, o_cb=None, encoding='utf-8')
    """ string/bytes, FfiAuthGranted*, [any], [function], [function], [encoding]
        char* app_id, FfiAuthGranted* auth_granted, void* user_data

        > return values of the callback functions:
        o_cb - App* app, void* user_data, FfiResult* result
    """

    @ffi.callback("void(void*)")
    def o_disconnect_notifier_cb(user_data):
        
        if o_disconnect_notifier_cb:
            o_disconnect_notifier_cb(user_data)
        else:
            pass

    @ffi.callback("void(App*, void*, FfiResult*)")
    def o_cb(app, user_data, result):
        
        if o_cb:
            o_cb(app, user_data, result)
        else:
            self.defaultFfiResult(result,'app_registered')

    app_id = ffi.new('char[]',self.toByteIfString(app_id,encoding))
    if user_data:
        user_data = ffi.new_handle(user_data)
    else:
        user_data = ffi.NULL

    self.lib.app_registered(app_id, auth_granted, user_data, o_disconnect_notifier_cb, o_cb)

Auth.app_registered = app_registered
del(app_registered)